In [ ]:
#pip install pdfplumber pandas openpyxl

  Using cached pandas-2.3.3-cp311-cp311-win_amd64.whl.metadata (19 kB)
  Using cached openpyxl-3.1.5-py2.py3-none-any.whl.metadata (2.5 kB)
  Using cached pillow-12.0.0-cp311-cp311-win_amd64.whl.metadata (9.0 kB)
  Using cached charset_normalizer-3.4.4-cp311-cp311-win_amd64.whl.metadata (38 kB)
  Using cached numpy-2.3.5-cp311-cp311-win_amd64.whl.metadata (60 kB)
  Using cached pytz-2025.2-py2.py3-none-any.whl.metadata (22 kB)
  Using cached et_xmlfile-2.0.0-py3-none-any.whl.metadata (2.7 kB)
  Using cached cffi-2.0.0-cp311-cp311-win_amd64.whl.metadata (2.6 kB)
  Using cached pycparser-2.23-py3-none-any.whl.metadata (993 bytes)
   ---------------------------------------- 0.0/5.6 MB ? eta -:--:--
   ---------------------------------------- 5.6/5.6 MB 49.2 MB/s  0:00:00
Using cached pandas-2.3.3-cp311-cp311-win_amd64.whl (11.3 MB)
Using cached openpyxl-3.1.5-py2.py3-none-any.whl (250 kB)
Using cached charset_normalizer-3.4.4-cp311-cp311-win_amd64.whl (106 kB)
   -------------------------

In [7]:
import pdfplumber
import pandas as pd
import re
import os

def procesar_horarios_v6_multi_docente(ruta_pdf):
    if not os.path.exists(ruta_pdf):
        print(f"Error: No encuentro el archivo: {ruta_pdf}")
        return None

    data_procesada = []
    
    # Contexto
    ctx = {
        "codigo": None, "curso": None, "creditos": None, 
        "prerequisitos": [], "observaciones": None, 
        "seccion": None, "docente": None
    }
    
    leyendo_prerequisitos = False
    palabras_clave = ["CLASE", "FINAL", "PARCIAL", "PRÁCTICA", "LABORATORIO", "EXAMEN", "RECUPERACIÓN"]

    print(f"Procesando (v6): {ruta_pdf}")
    
    with pdfplumber.open(ruta_pdf) as pdf:
        total_paginas = len(pdf.pages)
        
        for i, page in enumerate(pdf.pages):
            print(f"Pagina {i + 1}/{total_paginas}...", end="\r")
            
            table = page.extract_table()
            if not table: continue
                
            for row in table:
                row = [str(cell).strip() if cell else "" for cell in row]
                row_str = " ".join(row)
                col0 = row[0]
                
                if "Dirección de Asuntos" in row_str or "Horarios ofertados" in row_str: continue
                if "Secc" in col0 and "Docentes" in row_str: continue

                # ---------------------------------------------------------
                # 1. DETECCIÓN DE CURSO (Igual que antes)
                # ---------------------------------------------------------
                match_curso = re.search(r'(?<!PREREQUISITO: )(^|\s)([A-Z0-9]{6})\s?-\s?(.*)', row_str)
                es_inicio_curso = re.match(r'^[A-Z0-9]{6}\s?-', col0)
                
                if match_curso and es_inicio_curso:
                    raw_code = match_curso.group(2)
                    raw_name = match_curso.group(3).strip()
                    
                    # Limpieza Créditos
                    match_cred = re.search(r'(\d{1,2},\d{2})', raw_name)
                    if match_cred:
                        ctx["creditos"] = match_cred.group(1)
                        raw_name = raw_name.replace(match_cred.group(1), "")
                    
                    # Limpieza Prerrequisito Inline
                    prereq_inline = ""
                    if "PREREQUISITO" in raw_name:
                        parts = raw_name.split("PREREQUISITO")
                        raw_name = parts[0]
                        prereq_inline = "PREREQUISITO" + parts[1]
                    
                    ctx["codigo"] = raw_code
                    ctx["curso"] = raw_name.strip(" :,-")
                    ctx["prerequisitos"] = [prereq_inline] if prereq_inline else []
                    ctx["observaciones"] = None; ctx["seccion"] = None; ctx["docente"] = None
                    leyendo_prerequisitos = True
                    continue

                # ---------------------------------------------------------
                # 2. METADATA (Prerrequisitos)
                # ---------------------------------------------------------
                if leyendo_prerequisitos:
                    # Chequeo de fin de metada (Si empieza sección o clase)
                    es_seccion = (len(col0) > 0 and len(col0) <= 3 and col0 not in palabras_clave and not re.search(r'\d', col0))
                    es_clase = any(kw in row for kw in palabras_clave)
                    
                    if es_seccion or es_clase:
                        leyendo_prerequisitos = False
                    else:
                        match_cred = re.search(r'(\d{1,2},\d{2})', row_str)
                        if match_cred and not ctx["creditos"]:
                            ctx["creditos"] = match_cred.group(1)
                            row_str = row_str.replace(match_cred.group(1), "")

                        if text_has_content(row_str):
                             ctx["prerequisitos"].append(row_str)
                        continue

                # ---------------------------------------------------------
                # 3. DETECCIÓN DE SECCIÓN Y DOCENTES (LÓGICA MEJORADA)
                # ---------------------------------------------------------
                es_seccion = (len(col0) > 0 and len(col0) <= 3 and 
                              col0 not in palabras_clave and not re.search(r'\d', col0))
                
                if es_seccion:
                    ctx["seccion"] = col0
                    
                    # A. Construir el bloque de texto sucio (hasta donde empiece el horario)
                    texto_sucio_partes = []
                    for cell in row[1:]:
                        if cell in palabras_clave or re.match(r'\d{2}:\d{2}', cell): break
                        if cell: texto_sucio_partes.append(cell)
                    full_text = " ".join(texto_sucio_partes)
                    
                    # B. EXTRAER MÚLTIPLES DOCENTES
                    # Regex Mejorada: Acepta Mayúsculas o TitleCase, evita dígitos (fechas)
                    # Patrón: Palabras (sin numeros) + Coma + Espacio + Palabras (sin numeros)
                    regex_nombre = r'([A-ZÑÁÉÍÓÚ][A-Za-zÑÁÉÍÓÚñáéíóú\s\-\.]+,[\s]+[A-ZÑÁÉÍÓÚ][A-Za-zÑÁÉÍÓÚñáéíóú\s\-\.]+)'
                    
                    # Usamos finditer para encontrar TODOS los nombres
                    docentes_encontrados = []
                    matches = list(re.finditer(regex_nombre, full_text))
                    
                    for m in matches:
                        docente_limpio = m.group(1).strip()
                        # Verificación extra: que no sea una fecha disfrazada (aunque el regex evita numeros)
                        if len(docente_limpio) > 5:
                            docentes_encontrados.append(docente_limpio)
                    
                    # C. SEPARAR OBSERVACIONES
                    # Quitamos los nombres del texto original para ver qué sobra
                    obs_sucia = full_text
                    for doc in docentes_encontrados:
                        obs_sucia = obs_sucia.replace(doc, "")
                    
                    # Limpiamos la basura que queda (slash, comas sueltas, espacios)
                    obs_limpia = re.sub(r'^\s*[/\-,]\s*', '', obs_sucia) # Quitar separadores al inicio
                    obs_limpia = re.sub(r'\s*[/\-,]\s*$', '', obs_limpia) # Quitar separadores al final
                    obs_limpia = re.sub(r'\s{2,}', ' ', obs_limpia).strip()
                    
                    # Guardar en Contexto
                    if docentes_encontrados:
                        ctx["docente"] = " / ".join(docentes_encontrados)
                    else:
                        # Si no hay patrón de nombre pero hay texto, todo es observación (o docente mal formateado)
                        # Asumiremos observación si dice "Dictado", sino docente raw
                        if "Dictado" in full_text or "Clases" in full_text:
                            obs_limpia = full_text
                        else:
                            ctx["docente"] = full_text # Fallback
                            
                    if len(obs_limpia) > 2:
                        ctx["observaciones"] = obs_limpia
                    else:
                        ctx["observaciones"] = None

                # ---------------------------------------------------------
                # 4. HORARIOS (Igual que antes)
                # ---------------------------------------------------------
                tipo_actividad = None
                idx_tipo = -1
                
                for idx, cell in enumerate(row):
                    if cell in palabras_clave:
                        tipo_actividad = cell; idx_tipo = idx; break
                    elif any(kw in cell for kw in palabras_clave) and len(cell) > 5:
                        for kw in palabras_clave:
                            if kw in cell: tipo_actividad = kw; break
                
                if tipo_actividad and idx_tipo != -1:
                    datos = [x for x in row[idx_tipo+1:] if x != ""]
                    if len(datos) >= 2:
                        prereq_final = " ".join(ctx["prerequisitos"]).replace("PREREQUISITO:", "").strip(" ()")
                        
                        data_procesada.append({
                            "Codigo": ctx["codigo"],
                            "Curso": ctx["curso"],
                            "Creditos": ctx["creditos"],
                            "Seccion": ctx["seccion"],
                            "Docente": ctx["docente"],
                            "Observaciones": ctx["observaciones"],
                            "Tipo": tipo_actividad,
                            "Dia": datos[0],
                            "Inicio": datos[1],
                            "Fin": datos[2] if len(datos) > 2 else "",
                            "Aula": datos[-1] if len(datos) > 3 else "",
                            "Prerrequisitos": prereq_final
                        })

    return pd.DataFrame(data_procesada)

def text_has_content(text):
    return len(text.replace(",", "").strip()) > 3

# Ejecución
# ruta_archivo = r"pdfs\Oferta-Academica-extraord_CE-2026-12.12.pdf"
ruta_archivo = r"C:\Users\johnb\Documents\Github\MatriculaUp\pdfs\REGULAR_Oferta-Academica-2025-II_18.08_10.03am.pdf"
df_final = procesar_horarios_v6_multi_docente(ruta_archivo)

if df_final is not None and not df_final.empty:
    df_final.to_excel("Horarios_UP_V6_Perfecto.xlsx", index=False)
    print("¡Listo! Se corrigió el corte de nombres múltiples y las observaciones.")
else:
    print("Error: No data.")

Procesando (v6): C:\Users\johnb\Documents\Github\MatriculaUp\pdfs\REGULAR_Oferta-Academica-2025-II_18.08_10.03am.pdf
¡Listo! Se corrigió el corte de nombres múltiples y las observaciones.
